# Manipulating inputs & output

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [6]:
retrieve = {"context": retriever, "question": RunnablePassthrough()}



In [9]:
from langchain_core.runnables import RunnableParallel

retrieve = RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
retrieve = RunnableParallel(context=retriever, question=RunnablePassthrough())


In [10]:
retrieval_chain = retrieve | prompt | model | StrOutputParser()
retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

## Using itemgetter as shorthand

In [11]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "polish"})

'Harrison pracował w Kensho.'

## Parallelize steps

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!'),
 'poem': AIMessage(content='Majestic bear, roaming free in the woods,\nStrength and grace, misunderstood by the world.')}

## Parallelism

In [13]:
%%timeit

joke_chain.invoke({"topic": "bear"})

The slowest run took 23.64 times longer than the fastest. This could mean that an intermediate result is being cached.
15.5 s ± 9.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit

poem_chain.invoke({"topic": "bear"})

In [ ]:
%%timeit

map_chain.invoke({"topic": "bear"})